In [1]:
from datetime import date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import matplotlib.animation as animation
warnings.filterwarnings('ignore')
from src.data_processing.fetch_data import fetch_data_subgraph
from src.models.liquidity_distribution import LiquidityAnalyzer, Pool, LiquidityPeriphery
from src.models.backtest import LiquidityProvisionBacktester
pd.set_option("display.max_columns",100)
%load_ext autoreload
%autoreload 2

POOLS = [
    (3, "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", "v3 USDC/ETH 0.3%", ),
    (3, "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", "v3 USDC/ETH 0.05%", ),
]
API_KEY = "893d7471304c5edf436c8ba60781762c"
POOL_ADDRESS = "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640".lower()
START_DATE = date(2024, 8, 1)
END_DATE = date(2024, 9, 1)
pool_query = """query get_pools($pool_id: ID!) {
  pools(where: {id: $pool_id}) {
    tick
    sqrtPrice
    liquidity
    feeTier
    totalValueLockedUSD
    totalValueLockedETH
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""
# POOL_ADDRESS = "0xc7bBeC68d12a0d1830360F8Ec58fA599bA1b0e9b".lower()
variables = {'pool_id': POOL_ADDRESS}
data_pool = fetch_data_subgraph(API_KEY, pool_query, variables, data_key='pools', first_n=100, batch_size=1000)[0]
pool = Pool(pool_address=POOL_ADDRESS,
            token0=data_pool['token0']['symbol'],
            token1=data_pool['token1']['symbol'],
            decimals0=data_pool['token0']['decimals'],
            decimals1=data_pool['token1']['decimals'],
            fee_tier=data_pool['feeTier'],
            sqrt_price_x96=data_pool["sqrtPrice"])

Fetched 1 records


In [2]:
# Example usage
print(pool.token0, pool.token1)
df_liquidity = pd.read_parquet(
        "/Users/gnapsamuel/Documents/AMM/proteus-LP-backtesting/data/liquidity_data.parquet"
    )


USDC WETH


In [3]:
initial_capital_amount_0 = 1 #ETH
initial_capital_amount_1 = float(df_liquidity.iloc[0]['active_price_inverted'])
initial_capital_USD = initial_capital_amount_0*(initial_capital_amount_1/initial_capital_amount_0) + initial_capital_amount_1
tick_range = 50
rebalance_cost = 0.0001

In [4]:
current_tick = int(df_liquidity.iloc[0]['active_tick_adjusted'])
lower_tick = int(current_tick - tick_range)
upper_tick = int(current_tick + tick_range)

In [50]:
initial_capital_amount_0 = 1000
initial_capital_amount_1 = .5409

In [51]:
lower_tick = 200070
upper_tick = 202250
current_tick = 201101

In [52]:
sqrt_ratio_x96 = liquidity_calculator.get_sqrt_ratio_at_tick(current_tick)
sqrt_ratio_a_x96 = liquidity_calculator.get_sqrt_ratio_at_tick(lower_tick)
sqrt_ratio_b_x96 = liquidity_calculator.get_sqrt_ratio_at_tick(upper_tick)

In [53]:
amount0 = initial_capital_amount_0 * (10**pool.decimals0)
amount1 = initial_capital_amount_1 * (10**pool.decimals1)

In [55]:
liquidity0 = liquidity_calculator.get_liquidity_for_amount0(sqrt_ratio_x96, sqrt_ratio_b_x96, amount0)

In [56]:
liquidity0

416659080672654

In [57]:
416554837570579-liquidity0

-104243102075

In [30]:
liquidity_calculator = LiquidityPeriphery(pool)
sqrt_ratio_x96, sqrt_ratio_a_x96, sqrt_ratio_b_x96, amount0, amount1, liquidity = liquidity_calculator.get_liquidity_for_amounts(initial_capital_amount_0, initial_capital_amount_1, lower_tick, upper_tick, current_tick)

In [ ]:
position_liq

In [19]:
liquidity

250987612589.0

In [27]:
416554837570579-liquidity

416303849957990.0

In [11]:
df_liquidity_s = df_liquidity.loc[df_liquidity['block_number'].isin(df_liquidity.block_number.unique()[:10])]

In [13]:
df_liquidity_s['block_number'].nunique()

10

In [15]:
df_liquidity_s

,block_number,block_timestamp,cumulative_dollar_volume,date,datetime,block,active_tick,tick,liquidity,price,amount0,amount1,amount0_real,amount1_real,active_tick_adjusted,sb,sa,active_price,active_price_inverted,amount_locked,amount_locked_USDC
0,18909995,1704080591,2.002804e+07,2024-01-01,2024-01-01 03:43:11,18909995,199011,198510,7.536409e+18,2394.700859,1.843444e+11,7.701861e+19,0.000000,77.018615,199010.0,198010,199010,0.000439,2277.687846,175424.363256,3.257797e+07
1,18909995,1704080591,2.002804e+07,2024-01-01,2024-01-01 03:43:11,18909995,199011,198520,7.537610e+18,2392.307475,1.842816e+11,7.706941e+19,0.000000,77.069413,199010.0,198020,199020,0.000439,2277.687846,175540.064970,3.257797e+07
2,18909995,1704080591,2.002804e+07,2024-01-01,2024-01-01 03:43:11,18909995,199011,198530,8.531228e+18,2389.916483,2.084696e+11,8.727243e+19,0.000000,87.272430,199010.0,198030,199030,0.000439,2277.687846,198779.352993,3.257797e+07
3,18909995,1704080591,2.002804e+07,2024-01-01,2024-01-01 03:43:11,18909995,199011,198540,8.561083e+18,2387.527880,2.090945e+11,8.762163e+19,0.000000,87.621631,199010.0,198040,199040,0.000439,2277.687846,199574.724409,3.257797e+07
4,18909995,1704080591,2.002804e+07,2024-01-01,2024-01-01 03:43:11,18909995,199011,198550,9.407277e+18,2385.141665,2.296470e+11,9.633048e+19,0.000000,96.330476,199010.0,198050,199050,0.000439,2277.687846,219410.754242,3.257797e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,18917177,1704167867,2.007194e+08,2024-01-02,2024-01-02 03:57:47,18917177,198538,198990,1.839546e+19,2282.475776,4.392923e+11,1.925593e+20,439292.268649,0.000000,198530.0,198490,199490,0.000419,2388.005409,439292.268649,2.274335e+07
1006,18917177,1704167867,2.007194e+08,2024-01-02,2024-01-02 03:57:47,18917177,198538,199000,1.841580e+19,2280.194555,4.395580e+11,1.928685e+20,439557.990635,0.000000,198530.0,198500,199500,0.000419,2388.005409,439557.990635,2.274335e+07
1007,18917177,1704167867,2.007194e+08,2024-01-02,2024-01-02 03:57:47,18917177,198538,199010,1.232057e+19,2277.915614,2.939269e+11,1.290978e+20,293926.884098,0.000000,198530.0,198510,199510,0.000419,2388.005409,293926.884098,2.274335e+07
1008,18917177,1704167867,2.007194e+08,2024-01-02,2024-01-02 03:57:47,18917177,198538,199020,1.196945e+19,2275.638951,2.854077e+11,1.254814e+20,285407.728576,0.000000,198530.0,198520,199520,0.000419,2388.005409,285407.728576,2.274335e+07


In [14]:
backtester = LiquidityProvisionBacktester(pool, liquidity_calculator)
results_df = backtester.backtest_liquidity_provision(df_liquidity_s,
                                                     initial_capital_USD, 
                                                     initial_capital_amount_0,
                                                     initial_capital_amount_1,
                                                     tick_range)

2024-09-29 18:59:18,921 - LiquidityProvisionBacktester - INFO - Starting backtest with 9 blocks
2024-09-29 18:59:18,921 - LiquidityProvisionBacktester - INFO - Starting backtest with 9 blocks


Error at block 18909995: name 'capital_USD' is not defined
Error at block 18911563: name 'capital_USD' is not defined
Error at block 18912951: name 'capital_USD' is not defined
Error at block 18913979: name 'capital_USD' is not defined
Error at block 18914758: name 'capital_USD' is not defined
Error at block 18915118: name 'capital_USD' is not defined
Error at block 18915921: name 'capital_USD' is not defined
Error at block 18916179: name 'capital_USD' is not defined
Error at block 18916530: name 'capital_USD' is not defined
